In [7]:
# Importations nécessaires
import pandas as pd
import numpy as np # Ajouté pour la manipulation de tableaux si nécessaire
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
# MultiOutputClassifier n'est plus nécessaire pour cette approche en deux étapes
from sklearn.metrics import accuracy_score, classification_report
import joblib # Pour sauvegarder le modèle

# 1. Chargement des Données
file_path = r"donnees_agricoles_usa_traitees.csv" # Assurez-vous que ce fichier est dans le même dossier que le notebook ou ajustez le chemin
try:
    df_original = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Erreur : Le fichier {file_path} n'a pas été trouvé. Vérifiez le chemin.")
    exit()

print("Dataset original chargé (premières lignes) :")
print(df_original.head())
print("\n" + "="*50 + "\n")

# Copie pour modifications
df = df_original.copy()

# Suppression des colonnes 'Crop_Yield_MT_per_HA' et 'Economic_Impact_Million_USD' (selon la demande précédente)
colonnes_a_supprimer = ['Crop_Yield_MT_per_HA', 'Economic_Impact_Million_USD']
df = df.drop(columns=colonnes_a_supprimer, errors='ignore')

print(f"Dataset après suppression des colonnes {colonnes_a_supprimer} (premières lignes) :")
print(df.head())
print("\n" + "="*50 + "\n")

# 2. Définition des Variables d'entrée (X) et Cibles (y) initiales
features_columns = ['Average_Temperature_C', 'Total_Precipitation_mm', 'Soil_Health_Index']
crop_target_column_name = 'Crop_Type'
strategy_target_column_name = 'Adaptation_Strategies'

X_initial = df[features_columns]
y_crop_categorical = df[[crop_target_column_name]]
y_strategy_categorical = df[[strategy_target_column_name]]

# 3. Encodage des Variables Cibles Catégorielles
crop_type_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
adaptation_strategies_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Adapter et transformer les encodeurs
y_crop_encoded = crop_type_encoder.fit_transform(y_crop_categorical).ravel()
y_strategy_encoded = adaptation_strategies_encoder.fit_transform(y_strategy_categorical).ravel()

print("Variable cible 'Crop_Type' encodée (5 premières valeurs) :", y_crop_encoded[:5])
print("Noms des classes pour Crop_Type :", crop_type_encoder.categories_[0])
print("Variable cible 'Adaptation_Strategies' encodée (5 premières valeurs) :", y_strategy_encoded[:5])
print("Noms des classes pour Adaptation_Strategies :", adaptation_strategies_encoder.categories_[0])
print("\n" + "="*50 + "\n")

# 4. Division des Données
# Nous allons diviser X_initial et les deux y encodés en s'assurant que les index correspondent.
# D'abord, diviser X_initial et y_crop_encoded, en stratifiant sur y_crop_encoded pour une bonne répartition des classes.
X_train, X_test, y_train_crop, y_test_crop = train_test_split(
    X_initial, y_crop_encoded, test_size=0.25, random_state=42, stratify=y_crop_encoded
)

# Ensuite, utiliser les index de X_train et X_test pour obtenir les y_strategy correspondants
y_train_strat = y_strategy_encoded[X_train.index]
y_test_strat = y_strategy_encoded[X_test.index]


print(f"Taille de X_train : {X_train.shape}")
print(f"Taille de X_test : {X_test.shape}")
print(f"Taille de y_train_crop : {y_train_crop.shape}")
print(f"Taille de y_test_crop : {y_test_crop.shape}")
print(f"Taille de y_train_strat : {y_train_strat.shape}")
print(f"Taille de y_test_strat : {y_test_strat.shape}")
print("\n" + "="*50 + "\n")

# Note: La normalisation des features (X) a déjà été faite dans le script de nettoyage.
# Si vous aviez des features non normalisées ici, vous pourriez utiliser StandardScaler sur X_train et X_test.
# scaler = StandardScaler()
# X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
# X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

Dataset original chargé (premières lignes) :
   Average_Temperature_C  Total_Precipitation_mm  Soil_Health_Index Crop_Type  \
0                 0.5548                  0.2770             0.9644    Coffee   
1                 0.4925                  0.7408             0.6775    Barley   
2                 0.8186                  0.5268             0.9512  Soybeans   
3                 0.4294                  0.5174             0.3168     Wheat   
4                 0.8909                  0.5112             0.0261    Cotton   

     Adaptation_Strategies  Crop_Yield_MT_per_HA  Economic_Impact_Million_USD  
0         Water Management                0.3875                       0.1296  
1  Drought-resistant Crops                0.4596                       0.4806  
2            No Adaptation                0.6363                       0.6467  
3         Water Management                0.3182                       0.3896  
4            Crop Rotation                0.6282                    

In [8]:
# Configuration pour RandomForest - Étape 1: Crop_Type
rf_crop_estimator = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_crop_param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

print("Paramètres pour GridSearchCV (RandomForest - Crop_Type):")
print(rf_crop_param_grid)

Paramètres pour GridSearchCV (RandomForest - Crop_Type):
{'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]}


In [9]:
# GridSearchCV pour RandomForest - Étape 1: Crop_Type
print("Début du GridSearchCV pour RandomForest (Crop_Type)...")
grid_search_rf_crop = GridSearchCV(
    estimator=rf_crop_estimator,
    param_grid=rf_crop_param_grid,
    cv=3, # Réduire à 3 pour des tests plus rapides si nécessaire
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search_rf_crop.fit(X_train, y_train_crop)

print("\nGridSearchCV pour RandomForest (Crop_Type) terminé.")
print(f"Meilleurs paramètres : {grid_search_rf_crop.best_params_}")
print(f"Meilleur score de validation croisée (accuracy) : {grid_search_rf_crop.best_score_:.4f}")

# Sauvegarder le meilleur estimateur pour Crop_Type
best_rf_crop_model = grid_search_rf_crop.best_estimator_

Début du GridSearchCV pour RandomForest (Crop_Type)...
Fitting 3 folds for each of 27 candidates, totalling 81 fits

GridSearchCV pour RandomForest (Crop_Type) terminé.
Meilleurs paramètres : {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 50}
Meilleur score de validation croisée (accuracy) : 0.0995


In [ ]:
# Évaluation du meilleur modèle RandomForest pour Crop_Type sur X_test
y_pred_crop_rf = best_rf_crop_model.predict(X_test)

accuracy_crop_rf = accuracy_score(y_test_crop, y_pred_crop_rf)
print(f"\nAccuracy du RandomForest pour Crop_Type sur X_test : {accuracy_crop_rf:.4f}")

print("\nRapport de classification pour Crop_Type (RandomForest) sur X_test:")
# Décoder pour un rapport lisible
y_test_crop_decoded = crop_type_encoder.inverse_transform(y_test_crop.reshape(-1, 1)).ravel()
y_pred_crop_rf_decoded = crop_type_encoder.inverse_transform(y_pred_crop_rf.reshape(-1, 1)).ravel()
# S'assurer que tous les labels présents dans les données de test sont inclus
unique_labels_crop = np.unique(np.concatenate((y_test_crop_decoded, y_pred_crop_rf_decoded)))

print(classification_report(y_test_crop_decoded, y_pred_crop_rf_decoded, labels=unique_labels_crop, zero_division=0))

Début du GridSearchCV pour RandomForest (Crop_Type)...
Fitting 3 folds for each of 27 candidates, totalling 81 fits

GridSearchCV pour RandomForest (Crop_Type) terminé.
Meilleurs paramètres : {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 50}
Meilleur score de validation croisée (accuracy) : 0.0995


In [4]:
for model_name, best_model in best_estimators.items():
    if best_model is None:
        print(f"Pas de meilleur modèle trouvé pour {model_name} en raison d'une erreur.")
        continue

    print(f"Évaluation du meilleur modèle {model_name} sur l'ensemble de test...")
    y_pred = best_model.predict(X_test)
    
    y_pred_df = pd.DataFrame(y_pred, columns=target_columns, index=X_test.index)
    
    print(f"Précision (Accuracy) pour {model_name} par cible :")
    accuracy_crop_type = accuracy_score(y_test['Crop_Type'], y_pred_df['Crop_Type'])
    accuracy_adaptation_strategies = accuracy_score(y_test['Adaptation_Strategies'], y_pred_df['Adaptation_Strategies'])
    
    print(f"  Précision pour Crop_Type: {accuracy_crop_type:.4f}")
    print(f"  Précision pour Adaptation_Strategies: {accuracy_adaptation_strategies:.4f}")
    print("-" * 30)

    # Décodage pour des rapports plus lisibles
    y_test_decoded_crop = crop_type_encoder.inverse_transform(y_test[['Crop_Type']])
    y_pred_decoded_crop = crop_type_encoder.inverse_transform(y_pred_df[['Crop_Type']])
    
    y_test_decoded_strat = adaptation_strategies_encoder.inverse_transform(y_test[['Adaptation_Strategies']])
    y_pred_decoded_strat = adaptation_strategies_encoder.inverse_transform(y_pred_df[['Adaptation_Strategies']])
    
    print(f"Rapport de classification pour Crop_Type ({model_name}):")
    labels_crop_type = sorted(list(pd.Series(y_test_decoded_crop.ravel()).unique()))
    print(classification_report(y_test_decoded_crop, y_pred_decoded_crop, labels=labels_crop_type, zero_division=0))
    print("-" * 30)
    
    print(f"Rapport de classification pour Adaptation_Strategies ({model_name}):")
    labels_adaptation_strategies = sorted(list(pd.Series(y_test_decoded_strat.ravel()).unique()))
    print(classification_report(y_test_decoded_strat, y_pred_decoded_strat, labels=labels_adaptation_strategies, zero_division=0))
    
    print("\n" + "="*50 + "\n")

# Identifier le modèle globalement le meilleur basé sur le score de validation croisée
overall_best_model_name = None
highest_cv_score = -1

for model_name, results in grid_search_results.items():
    if 'best_score' in results and results['best_score'] > highest_cv_score:
        highest_cv_score = results['best_score']
        overall_best_model_name = model_name

if overall_best_model_name:
    print(f"Le meilleur modèle global basé sur le score CV est : {overall_best_model_name} avec un score de {highest_cv_score:.4f}")
    # Vous pouvez choisir de sauvegarder ce modèle spécifique
    final_best_model = best_estimators[overall_best_model_name]
    joblib.dump(final_best_model, f'best_agricultural_model_{overall_best_model_name}.joblib')
    joblib.dump(crop_type_encoder, 'crop_type_encoder.joblib')
    joblib.dump(adaptation_strategies_encoder, 'adaptation_strategies_encoder.joblib')
    print(f"Meilleur modèle ({overall_best_model_name}) et encodeurs sauvegardés.")
else:
    print("Aucun modèle n'a pu être entraîné avec succès.")


Évaluation du meilleur modèle RandomForest sur l'ensemble de test...
Précision (Accuracy) pour RandomForest par cible :
  Précision pour Crop_Type: 0.1146
  Précision pour Adaptation_Strategies: 0.2604
------------------------------
Rapport de classification pour Crop_Type (RandomForest):
              precision    recall  f1-score   support

      Barley       0.14      0.22      0.17         9
      Coffee       0.00      0.00      0.00         6
        Corn       0.15      0.18      0.17        11
      Cotton       0.17      0.11      0.13         9
      Fruits       0.25      0.11      0.15         9
        Rice       0.08      0.08      0.08        12
    Soybeans       0.12      0.08      0.10        12
   Sugarcane       0.00      0.00      0.00         8
  Vegetables       0.15      0.14      0.15        14
       Wheat       0.12      0.17      0.14         6

    accuracy                           0.11        96
   macro avg       0.12      0.11      0.11        96
weight